In [1]:
!pip install einops
!pip install snntorch

  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
Using cached einops-0.8.0-py3-none-any.whl (43 kB)


In [2]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
from scipy.io import loadmat
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import statistics
import os
import matplotlib.pyplot as plt
import scipy.stats
import math 
import sklearn
from sklearn import metrics
from snntorch import functional as SF
import datetime
import json

In [3]:
# RUN SETTINGS: 

# All parameters
num_inputs = 80 
num_outputs = 2
beta = .95 
batch_size = 32
num_epochs = 10
lr = 1e-3 
lr_decay = True 
lr_decay_gamma = 0.9 
# Dataset parameters
# filepaths for training and testing data 
root = "Datasets/rat_pd_sim_data"
training_data = "TestDataSet_1024"
testing_data = "VariedTrainingData"
num_steps = 150
num_steps_eval = 400 


# number of steps to hash spikes in data into. Should always be <= time steps of simulation data being loaded. 
# if smaller than simulation steps, data will be compressed into a smaller timeframe. This will speed up the model, but loses some timing granularity for neuron spikes. 
# if equal, no information is lost, and the model runs with exactly the same number of time steps as then rat simulation. 

# MOME parameters 
num_hidden = 144 # must be a perfects square. num_hidden = n^2 where n is an integer 
hidden_layers = 3 # This parameter is used ONLY for calculations, it will not effect network architecture. Should be set to correspond correctly to the networks actually created 
# keytopk^2 <= per_head_topk
key_topk = 10
per_head_topk = 64


# Comparison network settings 
normalize_params = True 
normalize_neurons = not normalize_params

# generate output folder: 

now = datetime.datetime.now()

timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

out_folder_name = f"figs/run_at_{timestamp}"
if not os.path.exists(out_folder_name):
    os.makedirs(out_folder_name)

def write_params(filename, params_dict): 
    with open(filename, 'w') as fp:
        json.dump(params_dict, fp)


In [4]:
def normalize(i, min, max):
    return (i - min) / (max - min)

# Hasher function used to compress spike times into our simulation time. Simple implimentation. 
# n: size of output array. Should equal to num_steps
# arr: The array to be hashed
# min/ max: Maximum and minimum values in input array. Max should be very slightly higher than the actually number, otherwise indexing errors can occur. 
# output: New hashed array of size n 
# Input is an array containing the 'time stamps' ie 1.01 for a spike occuring 1.01 seconds into simulation 
# This function than turns that spike into a '1' in an array of n indices, where i now becomes time 
def hasher(n, arr, min=0., max=2.0001):
    if len(arr) > n:
        print("There is too many items in this array to hash")
        print(f"The num_steps is {n} and the array length is {len(arr)}")
        print("Inserting an array full of ones instead")
        return np.ones(n)

    back_flag = False
    out = np.zeros(n)
    for item in arr: 
        norm = normalize(item, min, max)
        #print(norm)
        index = math.floor(norm * n)
        while out[index] == 1:
            if index == 0:
                back_flag = False
            if index == n-1:
                    back_flag = True

            if back_flag: 
                index -= 1 
                if index == 0:
                    back_flag = False
            else:
                index += 1 
                if index == n-1:
                    back_flag = True

        back_flag = False 
        
        out[index] = 1

    return out

In [5]:
# unpack_hash: 
# data: input the contents of one brain region from mat file 
# output: the contents of that file, now in in a singular array instead of nested lists and arrays 
# hashed via hash function above 

def unpack_hash(data):
    data = np.array(data[0])
    unpacked = [hasher(num_steps, item[0].flatten()) for item in data] 
    arr = np.array(unpacked)

    return np.fliplr(np.rot90(arr, -1))


In [6]:
#pass list of matfiles and elem to find maxlen of that attribute 
def max_len(mat_list, elem):
    max_elem = 0 
    for mat in mat_list:
        data = mat[elem]
        temp = max([len(item[0]) for item in data[0]])
        
        if(max_elem < temp):
            max_elem = temp 

    return max_elem

#pass list of matfiles and optional integer 'wiggle' 
#first, each matfile is passed through max_len function for each attribute of concern 
#wiggle is added to end of calculated maxlen. This is to ensure no problems arrive if testing data has some elements longer than training 
# TODO: some way to pass calculated maxlens to testdata will be important, so vector size is consistent between training and testing sets 
# then each file in mat_list is unpacked, and appended to each other to create one vector for each matfile 
# list of input vectors returned 

def constructor_hash(mat_list):
    out = []

    for mat in mat_list:
        TH_APs_data = unpack_hash(mat['TH_APs'])
        STNAPs_data = unpack_hash(mat['STN_APs'])
        GPe_APs_data = unpack_hash(mat['GPe_APs'])
        GPi_APs_data = unpack_hash(mat['GPi_APs'])
        Striat_APs_indr_data = unpack_hash(mat['Striat_APs_indr'])
        Striat_APs_dr_data = unpack_hash(mat['Striat_APs_dr'])
        Cor_APs_data = unpack_hash(mat['Cor_APs'])

        build = np.append(TH_APs_data, STNAPs_data, axis=1)
        build = np.append(build, GPe_APs_data, axis=1)
        build = np.append(build, GPi_APs_data, axis=1)
        build = np.append(build, Striat_APs_indr_data, axis=1)
        build = np.append(build, Striat_APs_dr_data, axis=1)
        build = np.append(build, Cor_APs_data, axis=1)
        out = out + [build]
        
    return np.array(out)

# pass dir = Directory containing samples 
# mat = name of the mat file you want to open 
def open_mat(dir, mat_name): 
    filename = os.path.join(dir, mat_name)
    mat = loadmat(filename)
    return mat

# input: list of mat files
# output: List of all gpi_alpha_beta_area's from mat files 

def get_beta_oscilation(mat_list):
    out = [] 
    for mat in mat_list: 
        out = out + [mat["gpi_alpha_beta_area"][0][0]]

    return out 

# dir = directory to load (ie Training Validation Set)
# os.loadir() loads all the filenames in a given directory
def load_dir(dir): 
    out = []
    bin_labels = [] 
    for mat_file in os.listdir(dir):
        if 'con' in mat_file:
            out = out + [open_mat(dir, mat_file)]
            bin_labels = bin_labels + [0]
        elif 'pd' in mat_file:
            out = out + [open_mat(dir, mat_file)]
            bin_labels = bin_labels + [1]
    
    alpha_beta = get_beta_oscilation(out) 
    out = constructor_hash(out)
    labels = list(zip(alpha_beta, bin_labels))
    
    return out, labels  


In [7]:
# set device
# NOTE: programmed on device without cuda. If device issues arise, ensure all relevant items properly cast to device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# set batch size


# Open directory specified, and cast build dataloader 
def build_dataloader(dir):
    full_dir, labels = load_dir(dir)
    data_tensor = torch.Tensor(full_dir).to(device)
    print(data_tensor.size())
    label_tensor = torch.Tensor(labels).to(device)
    dataset = torch.utils.data.TensorDataset(data_tensor, label_tensor)
    data = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return data 
# exploring_snn_architecture/Datasets/rat_pd_sim_data/VariedTrainingData
# exploring_snn_architecture/Datasets/rat_pd_sim_data/VariedTrainingData
data = build_dataloader(os.path.join(root, training_data))



torch.Size([1024, 150, 80])


In [8]:
data_list = list(data)
item, targets = data_list[0]
print(f"Shape of item tensor: {item.shape}")
print(f'This represents a batch size of {len(item)}, each sample in a batch is over {len(item[0])} timesteps, and has {len(item[0][0])} simulated cells\n')
print(f"Shape of targets tensor: {targets.shape}")
print(f'This represents a batch size of {len(item)}, each batch has a corresponding target label that is a tuple of size {len(item[0])} where index 0')
print('of that tuple is the alpha-beta oscillation and the index 1 of that tuple is a label 0 (no PD) or 1 (PD)')

Shape of item tensor: torch.Size([32, 150, 80])
This represents a batch size of 32, each sample in a batch is over 150 timesteps, and has 80 simulated cells

Shape of targets tensor: torch.Size([32, 2])
This represents a batch size of 32, each batch has a corresponding target label that is a tuple of size 150 where index 0
of that tuple is the alpha-beta oscillation and the index 1 of that tuple is a label 0 (no PD) or 1 (PD)


In [9]:
class Net_small(nn.Module):
    def __init__(self, num_inputs, num_hidden, beta, num_outputs, num_steps, batch_size):
        super().__init__()
        self.num_inputs = num_inputs 
        self.num_hidden = num_hidden 
        self.beta = beta 
        self.num_outputs = num_outputs 
        self.num_steps = num_steps
        self.batch_size = batch_size 

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_hidden)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden, num_hidden)
        self.lif3 = snn.Leaky(beta=beta)
        self.fc4 = nn.Linear(num_hidden, num_outputs)
        self.lif4 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        mem4 = self.lif4.init_leaky()
        
        # Record the final layer
        spk1_rec = []
        spk2_rec = []
        spk3_rec = []
        spk4_rec = []
        mem4_rec = [] 

        for step in range(self.num_steps):
            cur1 = self.fc1(x[0:self.batch_size, step, 0:self.num_inputs]) 
            spk1, mem1 = self.lif1(cur1, mem1)
            spk1_rec.append(spk1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)
            spk3_rec.append(spk3)
            cur4 = self.fc4(spk3)
            spk4, mem4 = self.lif4(cur4, mem4)
            spk4_rec.append(spk4)
            mem4_rec.append(mem4)
        

        spk1_rec = torch.stack(spk1_rec).detach().cpu() 
        spk2_rec = torch.stack(spk2_rec).detach().cpu() 
        spk3_rec = torch.stack(spk3_rec).detach().cpu() 
        spk4_rec = torch.stack(spk4_rec).detach().cpu() 
        
        return [spk1_rec, spk2_rec, spk3_rec, spk4_rec], mem4

In [10]:
from einops import einsum
import snntorch as snn 
import torch 
import torch.nn as nn 

# .memory source: https://github.com/facebookresearch/XLM/blob/main/xlm/model
from einops.layers.torch import Rearrange
import torch.nn.functional as F

DEBUG = False

def debug(input):
    if DEBUG: 
        print(input)
def debug_exit():
    assert(DEBUG==False)

class Printer(nn.Module): 
    def __init__(self): 
        super().__init__() 

    def forward(self, input): 
        print(input.size())
        return input

def exists(v):
    return v is not None

def default(v, d):
    return v if exists(v) else d

class RMSNorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.scale = dim ** 0.5
        self.gamma = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        return F.normalize(x, dim=-1) * self.scale * self.gamma

class ProductKeyMemory(nn.Module):
    def __init__(self, dim, num_keys):
        super().__init__()
        self.dim = dim
        self.num_keys = num_keys
        self.keys = nn.Parameter(torch.randn(num_keys, dim // 2))
        
    def forward(self, query):
        query = query.view(query.shape[0], 2, -1)
        dots = torch.einsum('bkd,nd->bkn', query, self.keys)
        return dots.view(query.shape[0], -1)

class PEER(nn.Module):
    def __init__(
        self,
        dim,
        *,
        heads=8,
        num_experts=1_000_000,
        num_experts_per_head=16,
        activation=nn.GELU,
        dim_key=None,
        product_key_topk=None,
        separate_embed_per_head=False,
        pre_rmsnorm=False,
        dropout=0.
    ):
        super().__init__()

        self.norm = RMSNorm(dim) if pre_rmsnorm else nn.Identity()

        self.heads = heads
        self.separate_embed_per_head = separate_embed_per_head
        self.num_experts = num_experts

        num_expert_sets = heads if separate_embed_per_head else 1

        self.weight_down_embed = nn.Embedding(num_experts * num_expert_sets, dim)
        self.weight_up_embed = nn.Embedding(num_experts * num_expert_sets, dim)

        self.activation = activation

        assert (num_experts ** 0.5).is_integer(), '`num_experts` needs to be a square'
        assert (dim % 2) == 0, 'feature dimension should be divisible by 2'

        dim_key = default(dim_key, dim // 2)
        self.num_keys = int(num_experts ** 0.5)

        self.to_queries = nn.Sequential(
            nn.Linear(dim, dim_key * heads * 2, bias=False),
            Rearrange('b (p d) -> p b d', p=2)
        )

        self.product_key_topk = default(product_key_topk, num_experts_per_head)
        self.num_experts_per_head = num_experts_per_head

        self.keys = nn.Parameter(torch.randn(heads, self.num_keys, 2, dim_key))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, membrain):
        x = self.norm(x)
        debug(f'x.shape: {x.shape}')
        queries = self.to_queries(x)
        debug(f'queries.size: {queries.size()}')
        debug(f'self.keys.size(): {self.keys.size()}')

        sim = einsum(queries, self.keys, 'p b d, h k p d -> p b h k')
        debug(f'sim.size() {sim.size()}')
        

        (scores_x, scores_y), (indices_x, indices_y) = [s.topk(self.product_key_topk, dim=-1) for s in sim]
        debug(f'scores_x.size() {scores_x.size()}')
        

        all_scores = scores_x.unsqueeze(-1) + scores_y.unsqueeze(-2)
        all_indices = indices_x.unsqueeze(-1) * self.num_keys + indices_y.unsqueeze(-2)
        debug(f'all_score.size() {all_scores.size()}')
        debug(f'all_indices.size() {all_indices.size()}')


        all_scores = all_scores.view(*all_scores.shape[:-2], -1)
        all_indices = all_indices.view(*all_indices.shape[:-2], -1)
        debug(f'all_score.size (post view) {all_scores.size()}')
        debug(f'all_indices.size (post view) {all_indices.size()}')
        debug(all_scores[-1])
        

        scores, pk_indices = all_scores.topk(self.num_experts_per_head, dim=-1)
        debug(f'scores.size {scores.size()}')
        debug(f'pk_indices.size {pk_indices.size()}')
        indices = all_indices.gather(-1, pk_indices)
        debug(f'indices.size {indices.size()}')

        debug_exit() 

        if self.separate_embed_per_head:
            head_expert_offsets = torch.arange(self.heads, device=x.device) * self.num_experts
            indices = indices + head_expert_offsets.view(1, 1, -1, 1)

        weights_down = self.weight_down_embed(pk_indices)
        weights_up = self.weight_up_embed(pk_indices)

        x = einsum(x, weights_down, 'b d, b n k d -> b n k')

        spikes, membrain = self.activation(x) # x is now spike outputs of the LIF neuron, membrain potentials are stored in membrain 
        x = self.dropout(spikes)

        x = x * F.softmax(scores, dim=-1)

        x = einsum(x, weights_up, 'b n k, b n k d -> b d')

        return x, membrain, spikes
    
    def init_leaky(self):
        return self.activation.init_leaky() 



'''
def forward(self, x):
    b, s = x.shape
    positions = torch.arange(s, device=x.device).unsqueeze(0).expand(b, s)
    
    x = self.token_embedding(x) + self.position_embedding(positions)
    
    for layer in self.layers:
        x = layer(x)
    
    x = self.layer_norm(x)
    logits = self.lm_head(x)
    return logits
'''
class MOME_net(nn.Module):
    def __init__(self, num_inputs, num_hidden, beta, num_outputs, num_steps, batch_size, key_topk=2, per_head_topk=4):
        super().__init__()
        self.num_steps = num_steps 
        self.batch_size = batch_size 
        self.num_inputs = num_inputs 

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.p1 = MOME_layer(num_hidden, num_hidden, beta=beta, key_topk=key_topk, per_head_topk=per_head_topk)
        self.p2 = MOME_layer(num_hidden, num_hidden, beta=beta, key_topk=key_topk, per_head_topk=per_head_topk)
        # self.p3 = MOME_layer(num_hidden, num_hidden, beta=beta)
        # self.p4 = MOME_layer(num_outputs, num_outputs, beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky() 
        mem2 = self.p1.init_leaky()
        mem3 = self.p2.init_leaky()
        mem4 = self.lif2.init_leaky() 
        # mem3 = self.p3.init_leaky()
        # mem4 = self.p4.init_leaky()
        
        # Record the final layer
        spk1_rec = []
        spk2_rec = []
        spk3_rec = []
        spk4_rec = []
        mem4_rec = [] 

        for step in range(self.num_steps):
            cur1 = self.fc1(x[0:self.batch_size, step, 0:self.num_inputs]) 
            spk1, mem1 = self.lif1(cur1, mem1)
            spk1_rec.append(spk1)
            x1, mem2, spk2 = self.p1(spk1, mem2)
            spk2_rec.append(spk2)
            x2, mem3, spk3 = self.p2(x1, mem3)
            spk3_rec.append(spk3)
            cur2 = self.fc2(x2)
            spk4, mem4 = self.lif2(cur2, mem4)
            spk4_rec.append(spk4)
            mem4_rec.append(mem4)
        

        spk1_rec = torch.stack(spk1_rec).detach().cpu() 
        spk2_rec = torch.stack(spk2_rec).detach().cpu() 
        spk3_rec = torch.stack(spk3_rec).detach().cpu() 
        spk4_rec = torch.stack(spk4_rec).detach().cpu() 
        
        return [spk1_rec, spk2_rec, spk3_rec, spk4_rec], mem4
    


# https://github.com/facebookresearch/XLM/blob/main/xlm/model/transformer.py 
# top k selection involves generating a k value for each expert, that allows evaluation of which expert is best for a specific task 
# this is the most complex portion of a mixture of experts model 
# Each MOME layer must make a selection of experts each time 
'''
        self,
        dim, -> num inputs 
        *,
        heads=8,
        num_experts=1_000_000,-> also num inputs? 
        num_experts_per_head=16, -> other num outputs 
        activation=nn.GELU,
        dim_key=None,
        product_key_topk=None, -> num outputs 
        separate_embed_per_head=False,
        pre_rmsnorm=False,
        dropout=0.
'''
class MOME_layer(nn.Module):
    def __init__(self, num_inputs: int, num_outputs: int, beta, key_topk=2, per_head_topk=4): 
        super().__init__()
        self.num_experts = num_inputs

        self.peer = PEER(num_inputs, heads=1, num_experts=self.num_experts, product_key_topk=key_topk, activation=snn.Leaky(beta=beta), num_experts_per_head=per_head_topk)

    def forward(self, data, mem):
        data, mem, spk = self.peer(data, mem)
        
        return data, mem, spk
    
    def init_leaky(self):
        return self.peer.init_leaky() 
    



In [11]:
def count_spike(spikes): 
    output = []
    unpacked = []
    for item in spikes: 
        # time * batch * neurons 
        # sum in time and neuron dimensions 
        unpacked.append(item.sum(0).sum(-1))
        #gives spike count by batch 
        #print(item.sum(0).sum(-1).size())
    for i in range(unpacked[0].size()[0]): 
        count = 0
        for item in unpacked:
            count += torch.tensor(item[i].item()) 
        
        output.append(count)
    return torch.stack(output)
    

In [12]:

def train_loop(data, model, num_epochs, loss, optimizer, scheduler):
    loss_hist = []
    dtype = torch.float
    loss_by_epoch = [] 
    spk_hist = [] 

    # Outer training loop
    for epoch in range(num_epochs):
        epoch_loss = [] 
        #iter_counter = 0

        # Minibatch training loop
        for i, (item, targets) in enumerate(iter(data)):
            targets = targets[0:batch_size, 1:2]
            targets = targets.flatten()

            targets = targets.type(torch.long)

            # forward pass
            model.train()
            spk_rec, mem_rec = model(item)
            spk_hist.append(spk_rec) 

            # initialize the loss & sum over time
            loss_val = torch.zeros((1), dtype=dtype, device=device)

            loss_val = torch.sum(loss(mem_rec, targets))

            # Gradient calculation + weight update
            optimizer.zero_grad()
            loss_val.backward(retain_graph=True)
            optimizer.step()

            # Store loss history for future plotting
            loss_hist.append(loss_val.item())
            epoch_loss.append(loss_val.item())

            # Calculate accuracy without SF.accuracy_rate
            pred = torch.argmax(mem_rec.cpu().detach(), dim=1)
            accuracy = metrics.accuracy_score(targets.cpu().detach(), pred.cpu().detach())

            # Print accuracy
            if loss_val.item() > 0.001:
                if i % 2 == 0:
                    print(f"Epoch {epoch}, Iteration {i}, Train Loss: {loss_val.item():.2f}, Accuracy: {accuracy * 100:.2f}%")
            else:
                if i % 20 == 0:
                    print(f"Epoch {epoch}, Iteration {i}, Train Loss: {loss_val.item():.2f}, Accuracy: {accuracy * 100:.2f}%")
        if scheduler is not None: 
            scheduler.step() 

        loss_by_epoch.append(epoch_loss)
    
    return (loss_hist, loss_by_epoch, spk_hist)
        




In [13]:
mome_net = MOME_net(num_inputs=num_inputs, num_hidden=num_hidden, beta=beta, num_outputs=num_outputs, num_steps=num_steps, batch_size=batch_size, key_topk=key_topk, per_head_topk=per_head_topk).to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mome_net.parameters(), lr=lr, betas=(0.9, 0.999))
scheduler = None
if lr_decay: 
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, lr_decay_gamma)
loss_hist_mome, loss_by_epoch_mome, spk_hist = train_loop(data, mome_net, num_epochs, loss, optimizer, scheduler)
mome_net.cpu() 

Epoch 0, Iteration 0, Train Loss: 0.93, Accuracy: 46.88%
Epoch 0, Iteration 2, Train Loss: 0.96, Accuracy: 34.38%
Epoch 0, Iteration 4, Train Loss: 0.79, Accuracy: 40.62%
Epoch 0, Iteration 6, Train Loss: 0.94, Accuracy: 37.50%
Epoch 0, Iteration 8, Train Loss: 0.69, Accuracy: 53.12%
Epoch 0, Iteration 10, Train Loss: 0.56, Accuracy: 71.88%
Epoch 0, Iteration 12, Train Loss: 0.60, Accuracy: 68.75%
Epoch 0, Iteration 14, Train Loss: 0.56, Accuracy: 71.88%
Epoch 0, Iteration 16, Train Loss: 0.53, Accuracy: 71.88%
Epoch 0, Iteration 18, Train Loss: 0.63, Accuracy: 62.50%
Epoch 0, Iteration 20, Train Loss: 0.42, Accuracy: 84.38%
Epoch 0, Iteration 22, Train Loss: 0.35, Accuracy: 90.62%
Epoch 0, Iteration 24, Train Loss: 0.32, Accuracy: 96.88%
Epoch 0, Iteration 26, Train Loss: 0.29, Accuracy: 100.00%
Epoch 0, Iteration 28, Train Loss: 0.31, Accuracy: 93.75%
Epoch 0, Iteration 30, Train Loss: 0.20, Accuracy: 100.00%
Epoch 1, Iteration 0, Train Loss: 0.17, Accuracy: 100.00%
Epoch 1, Iteratio

MOME_net(
  (fc1): Linear(in_features=80, out_features=144, bias=True)
  (lif1): Leaky()
  (p1): MOME_layer(
    (peer): PEER(
      (norm): Identity()
      (weight_down_embed): Embedding(144, 144)
      (weight_up_embed): Embedding(144, 144)
      (activation): Leaky()
      (to_queries): Sequential(
        (0): Linear(in_features=144, out_features=144, bias=False)
        (1): Rearrange('b (p d) -> p b d', p=2)
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (p2): MOME_layer(
    (peer): PEER(
      (norm): Identity()
      (weight_down_embed): Embedding(144, 144)
      (weight_up_embed): Embedding(144, 144)
      (activation): Leaky()
      (to_queries): Sequential(
        (0): Linear(in_features=144, out_features=144, bias=False)
        (1): Rearrange('b (p d) -> p b d', p=2)
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (fc2): Linear(in_features=144, out_features=2, bias=True)
  (lif2): Leaky()
)

In [14]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

In [15]:
from sympy import symbols, Eq, solve


if normalize_params: 
    params_mome = count_parameters(mome_net)
    #x = hidden_neurons_normalized_for_param_count
    #h = hidden_layers 
    #o = num_outputs
    #n = num_inputs
    #y = params_mome 
    # y == (n * x + x) + ((h - 1) * (x * x + x)) + (x * o + o)  
    
    # Define variables
    x, o, n, h, y = symbols('x o n h y')
    
    # Define the equation
    equation = Eq((n * x + x) + ((h - 1) * (x * x + x)) + (x * o + o), y)
    
    # Solve for x
    solution = solve(equation.subs({o: num_outputs, n: num_inputs, h: hidden_layers, y: params_mome}), x)
    real_solutions = [sol.evalf() for sol in solution if sol.is_real]
    hidden_neurons_normalized_for_param_count = round(max(real_solutions)) 
    print("Neurons for x:", hidden_neurons_normalized_for_param_count)
    
    # holy fuck this was annoying math omg 
    num_hidden_standard = hidden_neurons_normalized_for_param_count
else: 
    num_hidden_standard = num_hidden



Neurons for x: 244


In [ ]:
net = Net_small(num_inputs=num_inputs, num_hidden=num_hidden_standard, beta=beta, num_outputs=num_outputs, num_steps=num_steps, batch_size=batch_size).to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999))
scheduler = None
if lr_decay: 
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, lr_decay_gamma)
loss_hist, loss_by_epoch, spk_hist_base = train_loop(data, net, num_epochs, loss, optimizer, scheduler)
net.cpu()

Epoch 0, Iteration 0, Train Loss: 1.10, Accuracy: 43.75%
Epoch 0, Iteration 2, Train Loss: 0.65, Accuracy: 59.38%
Epoch 0, Iteration 4, Train Loss: 0.59, Accuracy: 84.38%
Epoch 0, Iteration 6, Train Loss: 0.40, Accuracy: 100.00%
Epoch 0, Iteration 8, Train Loss: 0.24, Accuracy: 100.00%
Epoch 0, Iteration 10, Train Loss: 0.12, Accuracy: 100.00%
Epoch 0, Iteration 12, Train Loss: 0.05, Accuracy: 100.00%
Epoch 0, Iteration 14, Train Loss: 0.01, Accuracy: 100.00%
Epoch 0, Iteration 16, Train Loss: 0.00, Accuracy: 100.00%
Epoch 0, Iteration 20, Train Loss: 0.00, Accuracy: 100.00%
Epoch 0, Iteration 22, Train Loss: 0.05, Accuracy: 96.88%
Epoch 0, Iteration 26, Train Loss: 0.00, Accuracy: 100.00%
Epoch 0, Iteration 28, Train Loss: 0.27, Accuracy: 90.62%
Epoch 1, Iteration 0, Train Loss: 0.02, Accuracy: 100.00%
Epoch 1, Iteration 2, Train Loss: 0.21, Accuracy: 96.88%
Epoch 1, Iteration 14, Train Loss: 0.00, Accuracy: 100.00%
Epoch 1, Iteration 20, Train Loss: 0.00, Accuracy: 100.00%
Epoch 1, I

In [ ]:
params_dict = {
    "beta": beta,
    "learning_rate": lr,
    "learning_rate_decay": lr_decay,
    "learning_rate_decay_gamma": lr_decay_gamma, 
    "batch_size": batch_size, 
    "num_epochs": num_epochs, 
    "hidden_width_mome": num_hidden,
    "hidden_width_standard": int(num_hidden_standard),
    "params_mome": count_parameters(mome_net), 
    "params_standard": count_parameters(net), 
    "hidden_layers": hidden_layers,
    "key_topk": key_topk, 
    "per_head_topk": per_head_topk, 
    "normalize_params": normalize_params,
    "normalize_neurons": normalize_neurons, 
    "training_data": training_data, 
    "testing_data": testing_data, 
}
print(params_dict) 
write_params(os.path.join(out_folder_name, "run_params.json"), params_dict) 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))



mome = [] 
for item in loss_by_epoch_mome: 
    mome.append(statistics.mean(item))
    
standard = [] 
for item in loss_by_epoch: 
    standard.append(statistics.mean(item))
    
X = range(len(standard)) 

X_axis = np.arange(len(X)) 
  
plt.bar(X_axis - 0.2, mome, 0.4, label = 'MOME', color="blue") 
plt.bar(X_axis + 0.2, standard, 0.4, label = 'Standard', color="red") 
  
plt.xticks(X_axis, X) 

# Adding labels
ax.set_xlabel('Epoch')
ax.set_ylabel('Average Loss')
ax.set_title('Average Loss by Epoch')
ax.set_ylim((0, .8))
ax.legend()
plt.savefig(os.path.join(out_folder_name, "Average_Loss_by_Epoch.png"))

In [ ]:
# plot loss over time
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(range(len(loss_hist_mome)), loss_hist_mome, c="red", label="MOME_snn")
ax.plot(range(len(loss_hist)), loss_hist, c="blue", label="standard_snn")
ax.set_ylim((0,.8))

# Adding labels
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss')
ax.set_title('Loss Over Time')
ax.legend()
plt.savefig(os.path.join(out_folder_name, "Loss_Over_Time.png"))

In [ ]:
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else np.zeros(len(d))
    return data[s<m]

In [ ]:
mean = np.mean(reject_outliers(np.array(loss_hist)))
print(mean)

In [ ]:
# loss curve 
fig, ax = plt.subplots(figsize=(8, 6))
x = range(len(loss_hist_mome))

z = np.polyfit(x, loss_hist_mome, 20)
p = np.poly1d(z)
ax.plot(x, loss_hist_mome, p(x))

# Adding labels
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss')
ax.set_title('Loss Curve with Fitted Polynomial MOME')
ax.set_ylim((0, .8))

plt.savefig(os.path.join(out_folder_name, "Loss_Curve_with_Fitted_Polynomial_MOME.png"))

In [ ]:
# loss curve 
fig, ax = plt.subplots(figsize=(8, 6))
x = range(len(loss_hist))

z = np.polyfit(x, loss_hist, 20)
p = np.poly1d(z)
ax.plot(x, loss_hist, p(x))

# Adding labels
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss')
ax.set_title('Loss Curve with Fitted Polynomial Standard')
ax.set_ylim((0, .8))

plt.savefig(os.path.join(out_folder_name, "Loss_Curve_with_Fitted_Polynomial_Standard.png"))

In [ ]:
def print_batch_accuracy(data, targets, net, train=False):
    output, _ = net(data.view(batch_size, -1))
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

In [ ]:
def eval_loop(data_eval, model):
    f1_score = []
    accuracy = []
    preds = [] 
    spike_counts = []
    net.eval()
    for item, targets in iter(data_eval):
        targets = targets[0:batch_size, 1:2]
        targets = targets.flatten()
        targets = targets.type(torch.long)

        #Get predictions
        spk_rec, mem_rec = model(item)
        spike_counts = spike_counts + count_spike(spk_rec).tolist()

        #Convert to binary 
        pred = torch.argmax(mem_rec.detach(), dim=1)
        # Calculate metrics 



        f1_score.append(metrics.f1_score(targets.detach().cpu(), pred.cpu()))
        accuracy.append(metrics.accuracy_score(targets.detach().cpu(), pred.cpu()))
        preds.extend(pred.tolist())

    net.train()
    return(spike_counts, preds, f1_score, accuracy)

In [ ]:
num_steps = num_steps_eval 
eval_set = build_dataloader(os.path.join(root, testing_data))

In [ ]:
net.to(device) 
spike_count, model_predictions, f1_score, accuracy = eval_loop(eval_set, net)
net.cpu() 

In [ ]:
mome_net.to(device) 
spike_count_mome, model_predictions_mome, f1_score_mome, accuracy_mome = eval_loop(eval_set, mome_net)
mome_net.cpu() 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(range(len(spike_count)), spike_count, color="blue", label="standard")
ax.plot(range(len(spike_count_mome)), spike_count_mome, color="red", label="MOME")

# Adding labels
ax.set_xlabel('Data Point')
ax.set_ylabel('Spike Count')
ax.set_title('Spike Count by Data Point')
ax.legend()
plt.savefig(os.path.join(out_folder_name, "Spike_Count_by_Data_Point.png"))

In [ ]:
pd_spike_counts = []
pd_indexes = [] 
con_spike_counts = []
con_indexes = []  
count = 0 


for i in range(len(spike_count)): 
    if model_predictions[i] == 1: 
        pd_spike_counts.append(spike_count[i])
        pd_indexes.append(count) 
    else: 
        con_spike_counts.append(spike_count[i])
        con_indexes.append(count)
    count += 1 

fig, ax = plt.subplots(figsize=(8, 6))
#plt.setp(ax, xticks=range(max(len(no_pd_labels), len(pd_labels))))
ax.scatter(pd_indexes, pd_spike_counts, color='red', label='PD')
ax.scatter(con_indexes, con_spike_counts, color='blue', label='No PD')

ax.set_xlabel('Sample Index')
ax.set_ylabel('Total Spikes')
ax.set_title('Comparison of Total Spike Counts in Model Between Samples With and Without PD')
ax.legend()
plt.savefig(os.path.join(out_folder_name, "Spike_Counts_in_Model_Between_Samples_With_and_Without_PD_standard.png"))

In [ ]:
pd_spike_counts = []
pd_indexes = [] 
con_spike_counts = []
con_indexes = []  
count = 0 


for i in range(len(spike_count_mome)): 
    if model_predictions_mome[i] == 1: 
        pd_spike_counts.append(spike_count_mome[i])
        pd_indexes.append(count) 
    else: 
        con_spike_counts.append(spike_count_mome[i])
        con_indexes.append(count)
    count += 1 

fig, ax = plt.subplots(figsize=(8, 6))
#plt.setp(ax, xticks=range(max(len(no_pd_labels), len(pd_labels))))
ax.scatter(pd_indexes, pd_spike_counts, color='red', label='PD')
ax.scatter(con_indexes, con_spike_counts, color='blue', label='No PD')

ax.set_xlabel('Sample Index')
ax.set_ylabel('Total Spikes')
ax.set_title('Comparison of Total Spike Counts in Model Between Samples With and Without PD MOME')
ax.legend()
plt.savefig(os.path.join(out_folder_name, "Spike_Counts_in_Model_Between_Samples_With_and_Without_PD_mome.png"))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
spike_count_average_by_epoch = [np.mean(spike_count[i:i+batch_size]) for i in range(0,len(spike_count),batch_size)]
ax.plot(range(len(spike_count_average_by_epoch)), spike_count_average_by_epoch, color="blue", label="standard")
spike_count_average_by_epoch_mome = [np.mean(spike_count_mome[i:i+batch_size]) for i in range(0,len(spike_count_mome),batch_size)]
ax.plot(range(len(spike_count_average_by_epoch_mome)), spike_count_average_by_epoch_mome, color="red", label="MOME")
# Adding labels
ax.set_xlabel('Batch')
ax.set_ylabel('Spike Count')
ax.set_title('Mean Spike Count by Batch')
ax.legend()
plt.savefig(os.path.join(out_folder_name, "Mean_Spike_Count_by_Batch.png"))

In [ ]:
mean = np.mean(spike_count_average_by_epoch)


In [ ]:
mean_mome = np.mean(spike_count_average_by_epoch_mome)


In [ ]:
spike_improvement = (mean - mean_mome) / mean

In [ ]:
# Plot f1 
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(range(len(f1_score)), f1_score, color="blue", label="standard")
ax.plot(range(len(f1_score_mome)), f1_score_mome, color="red", label="MOME")

# Adding labels
ax.set_xlabel('Batch')
ax.set_ylabel('F1 Score')
ax.set_title('F1 Score by Batch')
ax.legend()
plt.savefig(os.path.join(out_folder_name, "f1_by_Batch.png"))

In [ ]:
# Plot accuracy
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(range(len(accuracy)), accuracy, color="blue", label="standard")
ax.plot(range(len(accuracy_mome)), accuracy_mome, color="red", label="MOME")

# Adding labels
ax.set_xlabel('Batch')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy by Batch')
ax.legend()
plt.savefig(os.path.join(out_folder_name, "Accuracy_by_Batch.png"))